<div style="text-align: right;">
  <img src="https://raw.githubusercontent.com/exasol/ai-lab/refs/heads/main/assets/Exasol_Logo_2025_Dark.svg" style="width:200px; margin: 10px;" />
</div>

## Basic UDF with GPU Support

In this tutorial we will build a very simple UDF which uses `pytorch` to validate GPU detection.

## Open Secure Configuration Storage


In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

### Instantiate ScriptLanguagesContainer

The following cell creates an instance of class `ScriptLanguageContainer` from the notebook-connector,
which enables using the`exaslct` in the AI Lab in a convenient way.

In [ ]:
from exasol.nb_connector.slc import ScriptLanguageContainer
slc = ScriptLanguageContainer(secrets=ai_lab_config, name="gpu_slc")

### Connect to the database and activate the container

In [ ]:
from exasol.nb_connector.language_container_activation import open_pyexasol_connection_with_lang_definitions

conn = open_pyexasol_connection_with_lang_definitions(ai_lab_config, schema=ai_lab_config.db_schema, compression=True)

### Create a GPU enabled UDF

Exasol's UDF provide the script option `%perInstanceRequiredAcceleratorDevices` which allow the user to define the behavior of the database when executing UDF's. This option can have the following values:
* `None`: NO accelerator selected/required (implicit default for CPU processing)
* `GpuNvidia`: NVIDIA GPU accelerator required by the UDF. The invocation of the UDF will fail if GPU is unavailable or the Script Language Container is not suited for GPU's.
* `None|GpuNvidia`: NVIDIA GPU accelerator preferred but fallback to NO accelerator (CPU processing)

The following sample UDF requires GPU support by adding the line:
```
 %perInstanceRequiredAcceleratorDevices GpuNvidia;
```
and returns the string `"GPU Found"` if the GPU was detected in the UDF.

In [ ]:
import textwrap
conn.execute(textwrap.dedent(f"""
CREATE OR REPLACE {slc.language_alias} SCALAR SCRIPT
gpu_available()
RETURNS VARCHAR(10000) AS
 %perInstanceRequiredAcceleratorDevices GpuNvidia;
 %env PYTORCH_CUDA_DEBUG=1;
 %env CUDA_DEVICE_DEBUG=1;


import torch
import os
import subprocess

def run(ctx):
    #result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    #return result.stdout
    if torch.cuda.is_available():
        return "GPU Found"
    else:
        return "GPU Not Found"
/
"""))

res = conn.execute("SELECT gpu_available();")
for r in res:
    print(r[0])

Now, try out the other values for `%perInstanceRequiredAcceleratorDevices` and see what happens.

### Finish
Now you can continue with [Advanced UDF with GPU support](./advanced_udf_with_gpu.ipynb)